In [14]:
%load_ext autoreload
%autoreload 2

import os, codecs, string, random

import spacy
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
corpus_root = 'twitter-datasets/'

[nltk_data] Downloading package wordnet to /Users/macbook/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
from spellchecker import SpellChecker
spell = SpellChecker()
spell.correction('harware')

'hardware'

In [16]:
nlp = spacy.load("en_core_web_sm")

In [17]:
tweets = list()

for book_file in os.listdir(corpus_root):
    if ("train_pos.txt") in book_file or ("train_neg.txt") in book_file:
        print(book_file)
        with codecs.open(os.path.join(corpus_root,book_file),encoding="utf8") as f:
            tweets.append(f.readlines())

train_pos.txt
train_neg.txt


In [18]:
example = tweets[1][16]
print(example)
doc = nlp(example)
print(doc.text)
print("Tokenizing: \n")
tokens = [token.text for token in doc]
print(tokens)

print("\nRemoving stopwords and punctuations: \n")
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
stop_words = [token.text for token in doc if token.is_stop or token.is_punct]
tokens_nostop = [token.text for token in doc if not token.is_stop and not token.is_punct]
print("Removed: ", stop_words)
print("Kept: ", tokens_nostop)


print('\nLemmatization: \n')
for token in doc:
    if token.text != token.lemma_:
        print(token.text,'--->',token.lemma_)
lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
print(lemmas)

<user> it's annoying because i secretly find it so good ...

<user> it's annoying because i secretly find it so good ...

Tokenizing: 

['<', 'user', '>', 'it', "'s", 'annoying', 'because', 'i', 'secretly', 'find', 'it', 'so', 'good', '...', '\n']

Removing stopwords and punctuations: 

Removed:  ['it', "'s", 'because', 'i', 'it', 'so', '...']
Kept:  ['<', 'user', '>', 'annoying', 'secretly', 'find', 'good', '\n']

Lemmatization: 

it ---> -PRON-
's ---> be
it ---> -PRON-
['<', 'user', '>', 'annoying', 'secretly', 'find', 'good', '\n']


In [19]:
def spell_correction(sentence):
    doc=nlp(sentence)
    spell = SpellChecker()
    corrected=[spell.correction(token.text) for token in doc]
    return ' '.join(corrected)

In [25]:
print(spell_correction('expo precision point whiteboard eraser (8 473kf ) ( office product ergonomic eraser with designated grip . precisi ... <url>'))

expo precision point whiteboard eraser ( 8 473kf ) ( office product ergonomic eraser with designated grip . precise p. < usl >


In [25]:
def truncate_small_words(sentence , minimum_size):
    doc=nlp(sentence)
    truncated=[token.text for token in doc if len(token)>=minimum_size]
    print(truncated)
    return ' '.join(truncated)

In [26]:
print(truncate_small_words(example,3))

['user', 'annoying', 'because', 'secretly', 'find', 'good', '...']
user annoying because secretly find good ...


In [30]:
def remove_digits(sentence):
    doc=nlp(sentence)
    removed=[token.text for token in doc if not token.text.isdigit()]
    print(removed)
    return ' '.join(removed)

In [31]:
print(remove_digits(example))

['<', 'user', '>', 'it', "'s", 'annoying', 'because', 'i', 'secretly', 'find', 'it', 'so', 'good', '...', '\n']
< user > it 's annoying because i secretly find it so good ... 



In [61]:
def replace_by_antonym(sentence):
    doc=nlp(sentence)
    new=list()
    i=0
    
    while (i< len(doc)):    
        if doc[i].text=='not' and i+1<=len(doc):
            antonyms=list()
            for syn in wordnet.synsets(doc[i+1].text):
                for lemma in syn.lemmas():
                    for antonym in lemma.antonyms():
                        antonyms.append(antonym.name())
            print(antonyms)            
            if len(antonyms)==1:
                new.append(antonyms[0])
                i+=2
            else:
                new.append(doc[i].text)
                i+=1
        else:
            new.append(doc[i].text) 
            i+=1
    return ' '.join(new)   

In [68]:
replace_by_antonym('alex the neighbour is not happy today')

['unhappy']


'alex the neighbour is unhappy today'

In [5]:
my_stop = ['\n', '<', '>', 'url', 'user']
for i in range(2):
    for j in range(10,20):
        example = tweets[i][j]
        doc = nlp(example)
        lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        lemmas = [lem for lem in lemmas if lem not in my_stop]
        print(example)
        print(lemmas)
        print("\n")


introduction to programming with c + + ( 2nd edition this solid foundation in the basics of c + + programming will ... <url>

['introduction', 'programming', 'c', '+', '+', '2nd', 'edition', 'solid', 'foundation', 'basic', 'c', '+', '+', 'programming']


introduction to programming with c + + ( 2nd edition this solid foundation in the basics of c + + programming will ... <url>

['introduction', 'programming', 'c', '+', '+', '2nd', 'edition', 'solid', 'foundation', 'basic', 'c', '+', '+', 'programming']


introduction to programming with c + + ( 2nd edition this solid foundation in the basics of c + + programming will ... <url>

['introduction', 'programming', 'c', '+', '+', '2nd', 'edition', 'solid', 'foundation', 'basic', 'c', '+', '+', 'programming']


<user> i'm white . #aw

['white', 'aw']


<user> dan i love and miss you ! don't be sad #wheresthegeneral

['dan', 'love', 'miss', 'sad', 'wheresthegeneral']


so many wonderful building in dc but still miss you <user>

['wonderful', '

In [7]:
neg = []
pos = []
curr = neg
n0, n1 = len(tweets[0]), len(tweets[1])
N = n0 + n1
c = 0
for i in range(2):
    for example in tweets[i]:
        if c % 200 == 0: 
            print('{}% done'.format(100*c/N), end = '\r')
        doc = nlp(example)
        lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        lemmas = [lem for lem in lemmas if lem not in my_stop]
        curr.append(lemmas)
        c += 1
    curr = pos

KeyboardInterrupt: 

In [ ]:
print(neg[50:60])

In [ ]:
with open('lemmas_neg.txt', 'w') as f:
    for tweet in neg:
        f.write(' '.join(lemma for lemma in tweet) + '\n')
with open('lemmas_pos.txt', 'w') as f:
    for tweet in pos:
        f.write(' '.join(lemma for lemma in tweet) + '\n')